In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import gsw

Sigma grid and generated dz grid from OM4_025

In [ ]:
s2_75 = [1010, 1014.3034, 1017.8088, 1020.843, 1023.5566, 1025.813, 
    1027.0275, 1027.9114, 1028.6422, 1029.2795, 1029.852, 1030.3762, 
    1030.8626, 1031.3183, 1031.7486, 1032.1572, 1032.5471, 1032.9207, 
    1033.2798, 1033.6261, 1033.9608, 1034.2519, 1034.4817, 1034.6774, 
    1034.8508, 1035.0082, 1035.1533, 1035.2886, 1035.4159, 1035.5364, 
    1035.6511, 1035.7608, 1035.8661, 1035.9675, 1036.0645, 1036.1554, 
    1036.2411, 1036.3223, 1036.3998, 1036.4739, 1036.5451, 1036.6137, 
    1036.68, 1036.7441, 1036.8062, 1036.8526, 1036.8874, 1036.9164, 
    1036.9418, 1036.9647, 1036.9857, 1037.0052, 1037.0236, 1037.0409, 
    1037.0574, 1037.0738, 1037.0902, 1037.1066, 1037.123, 1037.1394, 
    1037.1558, 1037.1722, 1037.1887, 1037.206, 1037.2241, 1037.2435, 
    1037.2642, 1037.2866, 1037.3112, 1037.3389, 1037.3713, 1037.4118, 
    1037.475, 1037.6332, 1037.8104, 1038]

In [ ]:
def dz_f1(n, dz_min, total, power, precision):
    dz = np.empty(n)
    
    # initial profile
    for i in range(n):
        dz[i] = (i / (n - 1)) ** power
    
    # rescale to total depth and round to precision
    dz[:] = (total - n*dz_min) * (dz[:] / np.sum(dz))
    dz[:] = np.around(dz[:], decimals=precision)
    
    # adjust bottom
    dz[-1] += total - np.sum(dz[:] + dz_min)
    dz[-1] = np.around(dz[-1], decimals=precision)
    
    dz[:] += dz_min
    
    return dz

In [ ]:
dz_75 = dz_f1(75, 2, 4000, 4.5, 2)
max_int_depth = np.insert(dz_f1(75, 5, 8000, 1, 2).cumsum(), 0, 0)
max_lay_thick = dz_f1(75, 400, 31000, 0.1, 2)

# HyCOM1 Algorithm

- build current interface positions downward from thickness
- build layer pressure from reference pressure, averaged position of adjacent interfaces and a compressibility fraction
- calculate density in the column and adjust to enforce monotonicity
- calculate P1M_H2 interpolation polynomials for density
- interpolate the grid onto density (using Newton-Raphson iteration)

In [ ]:
# reference pressure
p_ref = 2000
# compressibility fraction
compr = 0.01
# maximum number of iterations to find interface position
max_iter = 8
# tolerance for finding the edge position
max_tol = 1e-12
# offset to deal with zero derivative in iteration
eps = 1e-6

In [ ]:
def hycom(h, sa, ct):
    """
    Build a HyCOM1 grid from thicknesses h, absolute salinity sa
    and conservative temperature ct.
    """
    
    # calculate interface positions, summing downward
    z = np.concatenate((np.zeros((1, h.shape[1])), h.cumsum(axis=0)), axis=0)
    # allocate the output positions
    # copy here so we have the correct surface and bottom
    z_new = z.copy()
    
    # calculate pressure
    p = p_ref + compr * ((z[1:,:] + z[:-1,:]) / 2 - p_ref)
    
    # calculate density
    r = gsw.rho(sa, ct, p)
    
    # enforce monotonicity, preserving the deepest density
    for k in range(r.shape[0]-1, 1, -1):
        # update all values above the bottom
        r[k-1,:] = np.minimum(r[k-1,:], r[k,:])
        
    # calculate interpolation edge values and coefficients
    e = np.empty(h.shape + (2,))
    for k in range(1, h.shape[0]):
        # skip leftmost cell
        # left edge of current cell
        e[k,:,0]  = (r[k-1,:]*h[k,:] + r[k,:]*h[k-1,:]) / (h[k,:] + h[k-1,:])
        # right edge of previous cell
        e[k-1,:,1] = e[k,:,0]
        
    # boundaries
    e[0,:,0]  = r[0,:]
    e[-1,:,1] = r[-1,:]
    
    # bound edge values with limiter
    for k in range(e.shape[0]):
        # handle boundaries
        if k == 0:
            k0 = k
            k1 = k
            k2 = k + 1
        elif k == e.shape[0] - 1:
            k0 = k - 1
            k1 = k
            k2 = k
        else:
            k0 = k - 1
            k1 = k
            k2 = k + 1
            
        # thicknesses
        h_l = h[k0,:]
        h_c = h[k1,:]
        h_r = h[k2,:]
        
        # value
        u_l = r[k0,:]
        u_c = r[k1,:]
        u_r = r[k2,:]
        
        # edges (before bounding)
        u0_l = e[k,:,0]
        u0_r = e[k,:,1]
        
        # slopes
        s_l = 2 * (u_c - u_l) / (h_c + 1e-30)
        s_c = 2 * (u_r - u_l) / (h_l + 2*h_c + h_r + 1e-30)
        s_r = 2 * (u_r - u_c) / (h_c + 1e-30)
        
        # NB: this is converted to work on all columns simultaneously
        slope = np.sign(s_c) * np.minimum(np.abs(s_l),
                                          np.abs(s_c),
                                          np.abs(s_r))
        # no slope at local extremum
        slope[s_l * s_r <= 0] = 0
            
        # convert to local coordinate system
        slope *= h_c / 2
        
        u0_l[(u_l - u0_l) * (u0_l - u_c) < 0] = \
            u_c - np.sign(slope) * np.minimum(np.abs(slope),
                                              np.abs(u0_l - u_c))
            
        u0_r[(u_r - u0_r) * (u0_r - u_c) < 0] = \
            u_c + np.sign(slope) * np.minimum(np.abs(slope),
                                              np.abs(u0_r - u_c))
            
        # bound by neighbouring cell means
        u0_l = np.maximum(np.minimum(u0_l, np.maximum(u_l, u_c)),
                          np.minimum(u_l, u_c))
        u0_r = np.maximum(np.minimum(u0_r, np.maximum(u_r, u_c)),
                          np.minimum(u_r, u_c))
        
        # save updated edge values
        e[k,:,0] = u0_l
        e[k,:,1] = u0_r
        
    # average discontinuous edge values
    for k in range(e.shape[0] - 1):
        u0_l = e[k,:,1]
        u0_r = e[k+1,:,0]
        
        u0_avg = (u0_l + u0_r) / 2
        e[k,:,1][u0_l != u0_r] = u0_avg
        e[k+1,:,0][u0_l != u0_r] = u0_avg
        
    # P1M constants
    c = np.empty_like(e)
    # x=0 value is left edge
    c[:,:,0] = e[:,:,0]
    # local slope given by difference of edge values
    c[:,:,1] = e[:,:,1] - e[:,:,0]
    
    # now we can actually perform the Newton-Raphson iteration
    # to find the interface positions corresponding with a target density
    # this is probably best implemented as a regular loop
    
    # loop over columns
    for j in range(h.shape[1]):
        # find the positions of all target values within the column
        # except the surface and very bottom
        for k, t in enumerate(s2_75[1:-1]):
            # check whether we're too light, at an interface, or too dense
            if t <= e[0,j,0]:
                # too light, set to surface position
                z_new[k+1,j] = z[0,j]
                continue
                
            # do we land between the right edge of one cell
            # and the left edge of the next?
            # (in practice, this is just asking if we're exactly at the
            # interface because of the averaging we did before)
            i = (t >= e[:-1,j,1]) & (t <= e[1:,j,0])
            if np.any(i):
                z_new[k+1,j] = z[np.where(i)[0],j]
                continue
                
            if t >= e[-1,j,1]:
                # too dense, set to bottom position
                z_new[k+1,j] = z[-1,j]
                continue
                
            # we must be inside a cell, so find out which one
            i = (t > e[:,j,0]) & (t < e[:,j,1])
            ki = np.where(i)[0]
            
            # set up Newton-Raphson
            xi0 = 0.5
            i = 1
            delta = 1e10
            
            while i < max_iter and abs(delta) < max_tol:
                # polynomial at guess
                num = c[ki,j,0] + c[ki,j,1]*xi0
                # gradient of interpolating function
                den = c[ki,j,1]
                
                delta = -num / den
                # update guess
                xi0 += delta
                
                # check whether new estimate is out of bounds
                if xi0 < 0:
                    xi0 = 0
                    if c[ki,j,1] == 0:
                        xi0 += eps
                        
                if xi0 > 1:
                    xi0 = 1
                    if c[ki,j,1] == 0:
                        xi0 -= eps
                        
                i += 1
                
            z_new[k+1,j] = z[ki,j] + xi0 * h[ki,j]
            
    # adjust new positions according to nominal depths
    z_nom = np.insert(dz_75.cumsum(), 0, 0)[:,np.newaxis]
    z_new = np.maximum(z_new, z_nom)
    # also bound by total depth
    z_new = np.minimum(z_new, z[[-1],:])
    
    # also also bound by maximum depth and thickness
    z_new[1:-1,:] = np.minimum(z_new[1:-1,:], max_int_depth[1:-1,np.newaxis],
                               z_new[:-2,:] + max_lay_thick[:-1,np.newaxis])
            
    return z_new

In [ ]:
hycom(np.ones((75,1)) * 4000 / 75, np.zeros((75, 1)), np.zeros((75, 1)))